In [238]:
#!conda install mpl_toolkits
# todo: Bar chart with Plotly Express, MAP
#from mpl_toolkits.basemap import Basemap as Basemap

In [239]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import sparse # shrink file size of df
# pw will calculate DISTANCES (0..N) BASED on cosine similarity
from sklearn.metrics.pairwise import pairwise_distances

from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.naive_bayes import MultinomialNB, GaussianNB
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

In [240]:
df_clean = pd.read_csv('./datasets/df_clean.csv')

In [241]:
df_clean.shape
df_regression = df_clean.copy()
#df_regression.info()

## Prepare datasets

## Organizing Data, Dealing with Missing Data

In [242]:
# not consider Institutions are identified by their religious affiliation 
#df_regression = df_regression[df_regression['relaffil'].isnull()]
#df_regression['relaffil'].fillna(value = 0, inplace=True)
#df_regression.shape

In [243]:
# Only consider Public and Private NonProfit Colleges DUE to large missing data
#df_regression= df_regression[df_regression['control'] != 3] 
df_regression.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7112 entries, 0 to 7111
Data columns (total 90 columns):
Unnamed: 0         7112 non-null int64
unitid             7112 non-null int64
instnm             7112 non-null object
city               7112 non-null object
stabbr             7112 non-null object
numbranch          7112 non-null int64
main               7112 non-null int64
highdeg            7112 non-null int64
preddeg            7112 non-null int64
control            7112 non-null int64
distanceonly       6668 non-null float64
tuitionfee_in      4021 non-null float64
tuitionfee_out     3793 non-null float64
tuitionfee_prog    2484 non-null float64
tuitfte            6644 non-null float64
avgfacsal          4245 non-null float64
relaffil           921 non-null float64
adm_rate_all       2250 non-null float64
satvr25            1238 non-null float64
satvr75            1238 non-null float64
satmt25            1238 non-null float64
satmt75            1238 non-null float64
satvrmid 

In [244]:
# if the admission rate is Nan, fill it with 100%
df_regression.adm_rate_all.fillna(value = 1.0, inplace=True)
# where admission rate is 1, fill sat scores the lowest score
mask = (df_regression['adm_rate_all'] == 1.0)
df_regression['satvr25'][mask] = df_regression['satvr25'].min()
df_regression['satvr75'][mask] = df_regression['satvr75'].min()
df_regression['satmt25'][mask] = df_regression['satmt25'].min()
df_regression['satmt75'][mask] = df_regression['satmt75'].min()
df_regression['satvrmid'][mask] = df_regression['satvrmid'].min()
df_regression['satmtmid'][mask] = df_regression['satmtmid'].min()
df_regression['sat_avg_all'][mask] = df_regression['sat_avg_all'].min()
df_regression['actcm75'][mask] = df_regression['actcm75'].min()
df_regression['actcm25'][mask] = df_regression['actcm25'].min()
df_regression['actcmmid'][mask] = df_regression['actcmmid'].min()
df_regression['actmtmid'][mask] = df_regression['actmtmid'].min()
df_regression['actenmid'][mask] = df_regression['actenmid'].min()
   

C:\Users\yingr\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\yingr\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\yingr\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\yingr\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyW

In [245]:

df_regression = df_regression[['instnm','stabbr','adm_rate_all',
                          'satvr25','satvr75',
                          'satmt25','satmt75',
                          'satvrmid','satmtmid','sat_avg_all'
                         ]]


In [246]:
df_regression.info()
df_regression.dropna(inplace=True)
df_regression.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7112 entries, 0 to 7111
Data columns (total 10 columns):
instnm          7112 non-null object
stabbr          7112 non-null object
adm_rate_all    7112 non-null float64
satvr25         6187 non-null float64
satvr75         6187 non-null float64
satmt25         6187 non-null float64
satmt75         6187 non-null float64
satvrmid        6187 non-null float64
satmtmid        6187 non-null float64
sat_avg_all     6396 non-null float64
dtypes: float64(8), object(2)
memory usage: 555.8+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6187 entries, 0 to 7110
Data columns (total 10 columns):
instnm          6187 non-null object
stabbr          6187 non-null object
adm_rate_all    6187 non-null float64
satvr25         6187 non-null float64
satvr75         6187 non-null float64
satmt25         6187 non-null float64
satmt75         6187 non-null float64
satvrmid        6187 non-null float64
satmtmid        6187 non-null float64
sat_avg_all   

In [247]:
outlier_to_drop = [828,1866]
df_regression.drop(index = outlier_to_drop, inplace=True)
df_regression.shape

(6185, 10)

In [221]:
#df_regression.to_csv('./datasets/df_regression.csv')

### Check Corr() for Regression

In [222]:
#sns.pairplot(df_regression, x_vars= ['adm_rate_all'], y_vars=df_regression.columns )

In [248]:
df_regression.corr()['adm_rate_all'][df_regression.corr()['adm_rate_all'].abs() > 0.5].sort_values(ascending=False)

adm_rate_all    1.000000
satvr75        -0.866263
satvrmid       -0.870340
satvr25        -0.871599
satmt75        -0.871832
sat_avg_all    -0.875796
satmtmid       -0.877033
satmt25        -0.879592
Name: adm_rate_all, dtype: float64

### REGRESSION

In [249]:
df_regression.columns

Index(['instnm', 'stabbr', 'adm_rate_all', 'satvr25', 'satvr75', 'satmt25',
       'satmt75', 'satvrmid', 'satmtmid', 'sat_avg_all'],
      dtype='object')

In [250]:
def result_chk(y_train, y_test, train_preds, test_preds):
    train_mse = mean_squared_error(y_train, train_preds)
    test_mse = mean_squared_error(y_test, test_preds)
    print ('Train MSE: {}'.format(train_mse) )
    print('Test MSE: {}'.format(test_mse) )
    train_rmse = mean_squared_error(y_train, train_preds) **(1/2)
    test_rmse = mean_squared_error(y_test, test_preds) **(1/2)
    print ('Train RMSE: {}'.format(train_rmse) )
    print('Test RMSE: {}'.format(test_rmse) )
    train_r2 = r2_score(y_train, train_preds)
    test_r2 = r2_score(y_test, test_preds)
    print ('Train R-squared: {}'.format(train_r2) )
    print('Test R-squared: {}'.format(test_r2) )
    print('cross_val_score: {}'.format(cross_val_score(lr, X_train, y_train, cv=5).mean()))

In [305]:
def run_lr(X_train, X_test, y_train, y_test):
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    train_preds =  lr.predict(X_train)
    test_preds = lr.predict(X_test)
    result_chk(y_train, y_test, train_preds, test_preds)
    return(lr)

In [306]:
def run_rf(X_train, X_test, y_train, y_test):
    rf = RandomForestRegressor(random_state=42)
    rf_params = {
        'n_estimators' : [60, 100, 150, 200], 
        'max_depth' : [None],
        'max_features' : [None, 'auto'],  # to use max_features
        'min_samples_leaf' : [5, 10, 20],
    }
    rf_gs = GridSearchCV(rf, param_grid=rf_params, cv=3)
    rf_gs.fit(X_train, y_train)
    print(f' Best Params: {rf_gs.best_params_}')
    train_preds =  rf_gs.predict(X_train)
    test_preds = rf_gs.predict(X_test)
    result_chk(y_train, y_test, train_preds, test_preds)
    return(rf_gs)

### Safety Category Colleges

In [302]:
#X= df_regreasion[['satvr25','satvr75','satmt25','satmt75']]
#X= df_regreasion[['satvr75','satmt75']] # Safety
#X= df_regreasion[['satvr25','satmt25','satvrmid', 'satmtmid',]] # reach
#X= df_regreasion[['satvrmid', 'satmtmid','sat_avg_all']] # target sat_avg_all
X= df_regression[['satvr75','satmt75']] # Safety
y= df_regression['adm_rate_all'] # .map(np.log)
X_train, X_test, y_train, y_test = train_test_split(X, y)

,satvr75,satmt75
6537,260.0,338.0
3673,260.0,338.0
1552,260.0,338.0
3000,260.0,338.0
6329,260.0,338.0
...,...,...
2105,260.0,338.0
2028,620.0,620.0
2683,260.0,338.0
3892,260.0,338.0


Linear Regression

In [307]:
lr_safe = run_lr(X_train, X_test, y_train, y_test)    

Train MSE: 0.0065528758487007465
Test MSE: 0.006531572285848339
Train RMSE: 0.080949835384025
Test RMSE: 0.08081814329621993
Train R-squared: 0.7537107343077563
Test R-squared: 0.7775035465576998
cross_val_score: 0.7490369561441372


Random Forest

In [308]:
rf_safe = run_rf(X_train, X_test, y_train, y_test)

 Best Params: {'max_depth': None, 'max_features': None, 'min_samples_leaf': 10, 'n_estimators': 60}
Train MSE: 0.004418980413617153
Test MSE: 0.00537512495709579
Train RMSE: 0.06647541209813711
Test RMSE: 0.07331524368844305
Train R-squared: 0.833913007615738
Test R-squared: 0.8168976492300006
cross_val_score: 0.7490369561441372


In [299]:
#X_pred= pd.DataFrame([[770,770,770,770]], columns=['satvr25','satmt25','satvrmid','satmtmid'])
X_pred= pd.DataFrame([[770,770]] )
X_pred

,0,1
0,770,770


In [310]:
lr_safe.predict(X_pred)[0], rf_safe.predict(X_pred)[0]

(0.4532928206514688, 0.12556446210905517)

In [332]:
adm_preds = rf_safe.predict(X_pred)[0] + (lr_safe.predict(X_pred)[0] - rf_safe.predict(X_pred)[0])/3.

In [333]:
df_regression[df_regression['adm_rate_all'] < (adm_preds)].sort_values(by='adm_rate_all', ascending=False).head(5)

,instnm,stabbr,adm_rate_all,satvr25,satvr75,satmt25,satmt75,satvrmid,satmtmid,sat_avg_all
830,Georgia Institute of Technology-Main Campus,GA,0.233959,370.0,730.0,720.0,790.0,550.0,755.0,1389.0
2967,Carnegie Mellon University,PA,0.221984,700.0,760.0,730.0,800.0,730.0,765.0,1506.0
2448,United States Merchant Marine Academy,NY,0.221563,580.0,620.0,630.0,660.0,600.0,645.0,1269.0
1621,Wellesley College,MA,0.220791,690.0,760.0,670.0,770.0,725.0,720.0,1448.0
3714,Washington and Lee University,VA,0.219982,680.0,740.0,670.0,750.0,710.0,710.0,1449.0


### Target Category Colleges

In [350]:
#X= df_regreasion[['satvr25','satvr75','satmt25','satmt75']]
#X= df_regreasion[['satvr75','satmt75']] # Safety
#X= df_regreasion[['satvr25','satmt25','satvrmid', 'satmtmid',]] # reach
#X= df_regreasion[['satvrmid', 'satmtmid','sat_avg_all']] # target sat_avg_all
X= df_regression[['satvrmid', 'satmtmid']] # target
y= df_regression['adm_rate_all'] # .map(np.log)
X_train, X_test, y_train, y_test = train_test_split(X, y)

Linear Regression

In [351]:
lr_safe = run_lr(X_train, X_test, y_train, y_test)    

Train MSE: 0.006181558420654084
Test MSE: 0.0066252463156280025
Train RMSE: 0.07862288738436209
Test RMSE: 0.0813956160713094
Train R-squared: 0.7711234192426012
Test R-squared: 0.7648656063928563
cross_val_score: 0.7685056501038237


Random Forest

In [352]:
rf_safe = run_rf(X_train, X_test, y_train, y_test)

 Best Params: {'max_depth': None, 'max_features': None, 'min_samples_leaf': 20, 'n_estimators': 200}
Train MSE: 0.004435708485471558
Test MSE: 0.005309601748839497
Train RMSE: 0.0666011147464632
Test RMSE: 0.07286701413423977
Train R-squared: 0.8357647501964584
Test R-squared: 0.8115587061927276
cross_val_score: 0.7685056501038237


In [353]:
X_pred= pd.DataFrame([[770,770]] )
X_pred

,0,1
0,770,770


In [354]:
lr_safe.predict(X_pred)[0], rf_safe.predict(X_pred)[0]

(0.383777716575776, 0.19243743895812063)

In [363]:
adm_preds = rf_safe.predict(X_pred)[0] #+ (lr_safe.predict(X_pred)[0] - rf_safe.predict(X_pred)[0])/6.

In [364]:
df_regression[df_regression['adm_rate_all'] < (adm_preds)].sort_values(by='adm_rate_all', ascending=False).head(5)

,instnm,stabbr,adm_rate_all,satvr25,satvr75,satmt25,satmt75,satvrmid,satmtmid,sat_avg_all
1131,University of Notre Dame,IN,0.189225,680.0,750.0,690.0,770.0,715.0,730.0,1486.0
229,University of California-Berkeley,CA,0.171076,630.0,720.0,630.0,760.0,675.0,695.0,1414.0
3615,Middlebury College,VT,0.170951,660.0,750.0,660.0,760.0,705.0,710.0,1446.0
232,University of California-Los Angeles,CA,0.160978,620.0,710.0,600.0,740.0,665.0,670.0,1387.0
1957,Washington University in St Louis,MO,0.159636,720.0,770.0,750.0,800.0,745.0,775.0,1508.0


### Reach Category Colleges

In [378]:
#X= df_regreasion[['satvr25','satvr75','satmt25','satmt75']]
#X= df_regreasion[['satvr75','satmt75']] # Safety
#X= df_regreasion[['satvr25','satmt25','satvrmid', 'satmtmid',]] # reach
#X= df_regreasion[['satvrmid', 'satmtmid','sat_avg_all']] # target sat_avg_all
X= df_regression[['satvr25','satmt25','sat_avg_all']] # reach
y= df_regression['adm_rate_all'] # .map(np.log)
X_train, X_test, y_train, y_test = train_test_split(X, y)

Linear Regression

In [379]:
lr_safe = run_lr(X_train, X_test, y_train, y_test)    

Train MSE: 0.006275551925643107
Test MSE: 0.005793406784552863
Train RMSE: 0.07921838123594238
Test RMSE: 0.07611443216994306
Train R-squared: 0.771709607260485
Test R-squared: 0.7833171497809391
cross_val_score: 0.7702630186936834


Random Forest

In [380]:
rf_safe = run_rf(X_train, X_test, y_train, y_test)

 Best Params: {'max_depth': None, 'max_features': None, 'min_samples_leaf': 10, 'n_estimators': 200}
Train MSE: 0.003822696333150092
Test MSE: 0.005028317677475546
Train RMSE: 0.06182795753662005
Test RMSE: 0.07091063162513465
Train R-squared: 0.8609389488671457
Test R-squared: 0.8119327285859866
cross_val_score: 0.7702630186936834


In [381]:
X_pred= pd.DataFrame([[770,770,1540]] )
X_pred

,0,1,2
0,770,770,1540


In [382]:
lr_safe.predict(X_pred)[0], rf_safe.predict(X_pred)[0]

(0.34699007447839214, 0.1187928161819641)

In [383]:
adm_preds = rf_safe.predict(X_pred)[0] #+ (lr_safe.predict(X_pred)[0] - rf_safe.predict(X_pred)[0])/5.

In [384]:
df_regression[df_regression['adm_rate_all'] < (adm_preds)].sort_values(by='adm_rate_all', ascending=False).head(5)

,instnm,stabbr,adm_rate_all,satvr25,satvr75,satmt25,satmt75,satvrmid,satmtmid,sat_avg_all
3388,Vanderbilt University,TN,0.108544,710.0,770.0,730.0,800.0,740.0,765.0,1515.0
3154,Swarthmore College,PA,0.107013,690.0,760.0,690.0,780.0,725.0,735.0,1473.0
2036,Dartmouth College,NH,0.104467,710.0,770.0,720.0,790.0,740.0,755.0,1483.0
268,Claremont McKenna College,CA,0.103638,660.0,740.0,680.0,770.0,700.0,725.0,1452.0
2498,Duke University,NC,0.098588,670.0,750.0,710.0,790.0,710.0,750.0,1488.0
